In [27]:
# First ensemble with NSL-KDD
# Parameters

#----------------------------------------------
# 0 for not using it as base learner
# 1 for using it as base learner

use_model_ada = 1 
use_model_dnn = 1 
use_model_mlp = 1 
use_model_lgbm = 1 
use_model_rf = 1 
use_model_svm = 1
use_model_knn = 1 
#----------------------------------------------
# 0 for training the model
# 1 for using the saved version of the model

load_model_ada = 0 
load_model_dnn = 0 
load_model_mlp = 0 
load_model_lgbm = 0 
load_model_rf = 0 
load_model_svm = 0
load_model_knn = 0 
#----------------------------------------------

# load_model_ada = 1
# load_model_dnn = 1 
# load_model_mlp = 1 
# load_model_lgbm = 1 
# load_model_rf = 1 
# load_model_svm = 1
# load_model_knn = 1 
#----------------------------------------------

#----------------------------------------------
# feature_selection_bit = 0 # OFF
feature_selection_bit = 1 # On
pick_prob = 0 # set equal one to choose the dataset with probabilities, set to 0 to choose one with the classes.
# pick_prob = 0




In [28]:
# Specify the name of the output text file
if feature_selection_bit == 0:

    if pick_prob == 0:
        output_file_name = "ensemble_level_01_all_features_classes.txt"
        with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)
        with open(output_file_name, "a") as f: print('----ensemble_level_01_all_features_classes--', file = f)

    elif pick_prob == 1:
        output_file_name = "ensemble_level_01_all_features_probabilites.txt"
        with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)
        with open(output_file_name, "a") as f: print('----ensemble_level_01_all_features_probabilites--', file = f)

elif feature_selection_bit == 1:
    if pick_prob == 0:
        output_file_name = "ensemble_level_01_feature_selection_classes.txt"
        with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)
        with open(output_file_name, "a") as f: print('----ensemble_level_01_feature_selection_classes--', file = f)
    elif pick_prob == 1:
        output_file_name = "ensemble_level_01_feature_selection_probabilites.txt"
        with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)
        with open(output_file_name, "a") as f: print('----ensemble_level_01_feature_selection_probabilites--', file = f)

In [29]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:
# importing required libraries
import numpy as np
import pandas as pd
import pickle # saving and loading trained model
from os import path


# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import (StandardScaler, OrdinalEncoder,LabelEncoder, MinMaxScaler, OneHotEncoder)
from sklearn.preprocessing import Normalizer, MaxAbsScaler , RobustScaler, PowerTransformer

# importing library for plotting
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from keras.layers import Dense # importing dense layer

from keras.layers import Input
from keras.models import Model
# representation of model layers
#from keras.utils import plot_model
from sklearn.metrics import confusion_matrix
import shap
import time

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score

import joblib
from sklearn.model_selection import train_test_split
import sklearn
from tabulate import tabulate



In [30]:
# column_features = ['dnn',
#                    'rf',
#                    'lgbm',
#                 #    'ada',
#                 #    'knn',
#                 #    'mlp',
#                 #    'svm',
#                 #    'cat',
#                    'xgb',
#                    'label']
# in order of importance
# DNN
# RF
# XGB
# LGBM

column_features = ['dnn',
                   'rf',
                   'lgbm',
                   'xgb',
               #    'ada',
                #    'knn',
                #    'mlp',
                #    'svm',
                #    'cat',
                   'label']
# Assuming df is your DataFrame
# if feature_selection_bit == 1:
#     df_level_00_1=pd.read_csv('base_models_prob_feature_selection.csv',names=column_features)
#     df_level_00_0=pd.read_csv('base_models_class_feature_selection.csv',names=column_features)

# if feature_selection_bit == 0:

#     df_level_00_1=pd.read_csv('base_models_prob_feature_selection.csv')
#     df_level_00_0=pd.read_csv('base_models_class_feature_selection.csv')

df_level_00_1=pd.read_csv('base_models_prob_feature_selection.csv')
df_level_00_0=pd.read_csv('base_models_class_feature_selection.csv')

if feature_selection_bit == 1:
    df_level_00_0 = df_level_00_0[column_features]
    df_level_00_1 = df_level_00_1[column_features]


In [31]:
df_level_00_1


,dnn,rf,lgbm,xgb,label
0,0.227917,0.991146,0.999975,0.996300,1.0
1,0.240160,0.991146,0.999979,0.996300,1.0
2,0.250123,0.991146,0.999971,0.996300,1.0
3,0.116905,0.563712,0.994721,0.951862,2.0
4,0.108086,0.828218,0.999755,0.977432,0.0
...,...,...,...,...,...
74254,0.239538,0.991146,0.999975,0.996300,1.0
74255,0.161583,0.958180,0.999924,0.989533,1.0
74256,0.264320,0.974195,0.999806,0.991595,0.0
74257,0.346290,0.466003,0.897302,0.734670,0.0


In [32]:
df_level_00_0

,dnn,rf,lgbm,xgb,label
0,0.0,1.0,1.0,1.0,1.0
1,0.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,2.0,2.0,2.0,2.0
4,3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
74254,1.0,1.0,1.0,1.0,1.0
74255,1.0,1.0,1.0,1.0,1.0
74256,0.0,0.0,0.0,0.0,0.0
74257,0.0,0.0,0.0,0.0,0.0


In [33]:
if pick_prob == 1:
    df_level_01 = df_level_00_1
else: 
    df_level_01 = df_level_00_0


y_01 = df_level_01.pop('label')
X_01 = df_level_01
df_level_01 = df_level_01.assign(label = y_01)


split = 0.7
X_train_01,X_test_01, y_train_01, y_test_01 = sklearn.model_selection.train_test_split(X_01, y_01, train_size=split)

In [34]:


# df_level_02 = pd.read_csv('base_models_class_feature_selection.csv')

# df_level_02

# y_02 = df_level_02.pop('label')
# X_02 = df_level_02
# df_level_02 = df_level_02.assign(label = y_01)


# split = 0.7
# X_train_02,X_test_02, y_train_02, y_test_02 = sklearn.model_selection.train_test_split(X_02, y_02, train_size=split)

## Training the stronger model - STACK level 01

In [35]:
X_test_01

,dnn,rf,lgbm,xgb
7603,0.0,0.0,0.0,0.0
6989,0.0,0.0,0.0,0.0
45131,1.0,1.0,1.0,1.0
17654,1.0,1.0,1.0,1.0
44943,1.0,2.0,2.0,2.0
...,...,...,...,...
62449,1.0,1.0,1.0,1.0
37605,1.0,1.0,1.0,1.0
10592,0.0,0.0,0.0,0.0
53255,0.0,0.0,3.0,3.0


### Voting

In [36]:
if pick_prob == 0:
    # Voting start

    import pandas as pd
    from scipy.stats import mode

    # Assuming 'df' is your original DataFrame with columns 'dnn', 'rf', 'lgbm', 'ada', 'knn', 'mlp', 'svm', 'label'
    df = X_test_01
    # Extract predictions columns
    
    # predictions = df[['dnn', 'rf', 'lgbm', 'ada', 'knn', 'mlp', 'svm','cat','xgb']]
        # selected_columns = df.loc[:, ~df.columns.isin(['rf'])]
    predictions = df.loc[:, ~df.columns.isin(['label'])] #df[column_features]

    # Use the mode function along axis 1 to get the most common prediction for each row
    ensemble_predictions, _ = mode(predictions.values, axis=1)

    # Add the ensemble predictions to the DataFrame
    df['ensemble'] = ensemble_predictions.astype(int)

    # Display the DataFrame with ensemble predictions
    print(df)

    pred_label = df ['ensemble'].values
    df.pop('ensemble')

    with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Voting', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    voting_acc_01 = Acc
    voting_pre_01 = Precision
    voting_rec_01 = Recall
    voting_f1_01 = F1
    voting_bacc_01 = BACC
    voting_mcc_01 = MCC
    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)

   


       dnn   rf  lgbm  xgb  ensemble
7603   0.0  0.0   0.0  0.0         0
6989   0.0  0.0   0.0  0.0         0
45131  1.0  1.0   1.0  1.0         1
17654  1.0  1.0   1.0  1.0         1
44943  1.0  2.0   2.0  2.0         2
...    ...  ...   ...  ...       ...
62449  1.0  1.0   1.0  1.0         1
37605  1.0  1.0   1.0  1.0         1
10592  0.0  0.0   0.0  0.0         0
53255  0.0  0.0   3.0  3.0         0
30562  1.0  2.0   0.0  0.0         0

[22278 rows x 5 columns]
---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0    3.0  4.0
0.0  11274.0    62.0    73.0   19.0  0.0
1.0    524.0  7567.0    17.0    3.0  0.0
2.0    157.0   175.0  1819.0    0.0  0.0
3.0    456.0     1.0     6.0  111.0  0.0
4.0     12.0     0.0     0.0    2.0  0.0
---------------------------------------------------------------------------------
METRICS
-------------


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### Average

In [37]:
# len(~df.columns.isin(['label']))

In [38]:
if pick_prob == 0:
    # Average start

    import pandas as pd
    from scipy.stats import mode

    # Assuming 'df' is your original DataFrame with columns 'dnn', 'rf', 'lgbm', 'ada', 'knn', 'mlp', 'svm', 'label'
    df = X_test_01
    # Extract predictions columns
    
    # predictions = df[['dnn', 'rf', 'lgbm', 'ada', 'knn', 'mlp', 'svm','cat','xgb']]
        # selected_columns = df.loc[:, ~df.columns.isin(['rf'])]
    predictions = df.loc[:, ~df.columns.isin(['label'])] #df[column_features]

   

    column_sums = df.sum(axis=1)
    # print(column_sums)
    # df['column_sums'] = column_sums
    # print(len(~df.columns.isin(['label'])))
    
       # Calculate the average sum along axis 1
    row_average = df.mean(axis=1)

    # Approximate the result to the closest integer
    rounded_average = row_average.round().astype(int)

    # print(rounded_average)

    df['results'] = rounded_average
    print(df)
    # Use the mode function along axis 1 to get the most common prediction for each row
    # ensemble_predictions, _ = mode(predictions.values, axis=1)

    # Add the ensemble predictions to the DataFrame
    # df['ensemble'] = ensemble_predictions.astype(int)/len(~df.columns.isin(['label']))
    # df['ensemble'] = column_sums / len(~df.columns.isin(['label']))

 

    #rules

    # for i in range(0,len(df)):
    #     if 


    # print()

    # Display the DataFrame with ensemble predictions
 
    pred_label = df ['results'].values

    # pred_label = df ['ensemble'].values
    # df.pop('ensemble')
    df.pop('results')

    # df.pop('column_sums')

    with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Average', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    avg_acc_01 = Acc
    avg_pre_01 = Precision
    avg_rec_01 = Recall
    avg_f1_01 = F1
    avg_bacc_01 = BACC
    avg_mcc_01 = MCC
    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)

   


       dnn   rf  lgbm  xgb  results
7603   0.0  0.0   0.0  0.0        0
6989   0.0  0.0   0.0  0.0        0
45131  1.0  1.0   1.0  1.0        1
17654  1.0  1.0   1.0  1.0        1
44943  1.0  2.0   2.0  2.0        2
...    ...  ...   ...  ...      ...
62449  1.0  1.0   1.0  1.0        1
37605  1.0  1.0   1.0  1.0        1
10592  0.0  0.0   0.0  0.0        0
53255  0.0  0.0   3.0  3.0        2
30562  1.0  2.0   0.0  0.0        1

[22278 rows x 5 columns]
---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0  3.0  4.0
0.0  10814.0   516.0    98.0  0.0  0.0
1.0    507.0  7579.0    25.0  0.0  0.0
2.0     61.0   203.0  1887.0  0.0  0.0
3.0     55.0   144.0   375.0  0.0  0.0
4.0      6.0     4.0     4.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
-------------------------------------


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


## Weighed Average

In [39]:
# import pandas as pd

# # Assuming df is your DataFrame
# data = {'dnn': [1.0, 0.0], 'rf': [1.0, 0.0], 'lgbm': [1.0, 0.0], 'xgb': [0.0, 0.0]}
# weights = {'dnn': 0.4, 'rf': 0.3, 'lgbm': 0.2, 'xgb': 0.1}
# df = pd.DataFrame(data)

# # Calculate the weighted average along axis 1
# weighted_average = (df.multiply(weights)).sum(axis=1) / sum(weights.values())

# # Approximate the result to the closest integer
# rounded_weighted_average = weighted_average.round().astype(int)

# print(rounded_weighted_average)


In [40]:
# selected_col = ['xgb','lgbm']

In [41]:
# df[['rf', 'dnn','lgbm']]
# # df[['rf', 'dnn']]
# df = df[selected_col]

In [42]:
# df

,xgb,lgbm
7603,0.0,0.0
6989,0.0,0.0
45131,1.0,1.0
17654,1.0,1.0
44943,2.0,2.0
...,...,...
62449,1.0,1.0
37605,1.0,1.0
10592,0.0,0.0
53255,3.0,3.0


In [43]:
# df

,xgb,lgbm
7603,0.0,0.0
6989,0.0,0.0
45131,1.0,1.0
17654,1.0,1.0
44943,2.0,2.0
...,...,...
62449,1.0,1.0
37605,1.0,1.0
10592,0.0,0.0
53255,3.0,3.0


In [44]:
# column_features
# move this up with column_features

#important update this as you need to select the important features,
#  the left is the least important while the right is the most important
# needs automation
feature_selection_columns_in_order_of_importance = [ 'xgb','lgbm', 'rf', 'dnn']

In [45]:
if pick_prob == 0:
    # Average start

    import pandas as pd
    from scipy.stats import mode

    # Assuming 'df' is your original DataFrame with columns 'dnn', 'rf', 'lgbm', 'ada', 'knn', 'mlp', 'svm', 'label'
    # df = X_test_01
    df = X_test_01[feature_selection_columns_in_order_of_importance]
    # Extract predictions columns
    
    # predictions = df[['dnn', 'rf', 'lgbm', 'ada', 'knn', 'mlp', 'svm','cat','xgb']]
        # selected_columns = df.loc[:, ~df.columns.isin(['rf'])]
    predictions = df.loc[:, ~df.columns.isin(['label'])] #df[column_features]

    # weight
    weights_values = []

    # linear weight distribution
    for i in range(0,len(~df.columns.isin(['label']))):
        weights_values.append(i/(len(~df.columns.isin(['label']))-1))
    print(weights_values)
    # weights_values = [10,3,2,2.3]
    print(weights_values)
    print(df)
    weighted_average = df.multiply(weights_values).sum(axis=1) / sum(weights_values)
    print(weighted_average)

    # weights = {'dnn': 0.4, 'rf': 0.3, 'lgbm': 0.2, 'xgb': 0.1}


    # Calculate the weighted average along axis 1
    # weighted_average = (df.multiply(weights)).sum(axis=1) / sum(weights.values())

    # Approximate the result to the closest integer
    rounded_weighted_average = weighted_average.round().astype(int)

    print(rounded_weighted_average)

    # print(rounded_average)

    df['results'] = rounded_average
    print(df)
    # Use the mode function along axis 1 to get the most common prediction for each row
    # ensemble_predictions, _ = mode(predictions.values, axis=1)

    # Add the ensemble predictions to the DataFrame
    # df['ensemble'] = ensemble_predictions.astype(int)/len(~df.columns.isin(['label']))
    # df['ensemble'] = column_sums / len(~df.columns.isin(['label']))

 

    #rules

    # for i in range(0,len(df)):
    #     if 


    # print()

    # Display the DataFrame with ensemble predictions
 
    pred_label = df ['results'].values

    # pred_label = df ['ensemble'].values
    # df.pop('ensemble')
    df.pop('results')

    # df.pop('column_sums')

    with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Weighed Average', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    weighed_avg_acc_01 = Acc
    weighed_avg_pre_01 = Precision
    weighed_avg_rec_01 = Recall
    weighed_avg_f1_01 = F1
    weighed_avg_bacc_01 = BACC
    weighed_avg_mcc_01 = MCC
    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)

   


[0.0, 0.3333333333333333, 0.6666666666666666, 1.0]
[0.0, 0.3333333333333333, 0.6666666666666666, 1.0]
       xgb  lgbm   rf  dnn
7603   0.0   0.0  0.0  0.0
6989   0.0   0.0  0.0  0.0
45131  1.0   1.0  1.0  1.0
17654  1.0   1.0  1.0  1.0
44943  2.0   2.0  2.0  1.0
...    ...   ...  ...  ...
62449  1.0   1.0  1.0  1.0
37605  1.0   1.0  1.0  1.0
10592  0.0   0.0  0.0  0.0
53255  3.0   3.0  0.0  0.0
30562  0.0   0.0  2.0  1.0

[22278 rows x 4 columns]
7603     0.000000
6989     0.000000
45131    1.000000
17654    1.000000
44943    1.500000
           ...   
62449    1.000000
37605    1.000000
10592    0.000000
53255    0.500000
30562    1.166667
Length: 22278, dtype: float64
7603     0
6989     0
45131    1
17654    1
44943    2
        ..
62449    1
37605    1
10592    0
53255    0
30562    1
Length: 22278, dtype: int64
       xgb  lgbm   rf  dnn  results
7603   0.0   0.0  0.0  0.0        0
6989   0.0   0.0  0.0  0.0        0
45131  1.0   1.0  1.0  1.0        1
17654  1.0   1.0  1.0  1.0 


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


## bagging  with DT

In [90]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# # Load the Iris dataset
# iris = load_iris()
# X, y = iris.data, iris.target

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define the base classifier (Decision Tree in this case)
base_classifier = DecisionTreeClassifier(random_state=42)

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train_01, y_train_01)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test_01)

# Evaluate accuracy
# accuracy = accuracy_score(y_test_01, y_pred)
# print(f'Accuracy: {accuracy}')

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

with open(output_file_name, "a") as f: print('Bagging with DT', file = f)


print('---------------------------------------------------------------------------------')
print('CONFUSION MATRIX')
print('---------------------------------------------------------------------------------')


pred_label = y_pred

confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
all_unique_values = sorted(set(pred_label) | set(y_test_01))
z = np.zeros((len(all_unique_values), len(all_unique_values)))
rows, cols = confusion_matrix.shape
z[:rows, :cols] = confusion_matrix
confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
# confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
# with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
print(confusion_matrix)
with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)
TP_total = sum(TP)
TN_total = sum(TN)
FP_total = sum(FP)
FN_total = sum(FN)

TP_total = np.array(TP_total,dtype=np.float64)
TN_total = np.array(TN_total,dtype=np.float64)
FP_total = np.array(FP_total,dtype=np.float64)
FN_total = np.array(FN_total,dtype=np.float64)



#----------------------------------------------------------------#----------------------------------------------------------------

print('---------------------------------------------------------------------------------')
print('METRICS')
print('---------------------------------------------------------------------------------')


Acc = accuracy_score(y_test_01, pred_label)
Precision = precision_score(y_test_01, pred_label, average='macro')
Recall = recall_score(y_test_01, pred_label, average='macro')
F1 =  f1_score(y_test_01, pred_label, average='macro')
BACC = balanced_accuracy_score(y_test_01, pred_label)
MCC = matthews_corrcoef(y_test_01, pred_label)


bag_dt_acc_01 = Acc
bag_dt_pre_01 = Precision
bag_dt_rec_01 = Recall
bag_dt_f1_01 = F1
bag_dt_bacc_01 = BACC
bag_dt_mcc_01 = MCC
# with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
print('Accuracy total: ', Acc)
print('Precision total: ', Precision )
print('Recall total: ', Recall )
print('F1 total: ', F1 )
print('BACC total: ', BACC)
print('MCC total: ', MCC)

with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)




---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0    3.0  4.0
0.0  11214.0    69.0    54.0   91.0  0.0
1.0     51.0  8052.0     4.0    4.0  0.0
2.0     45.0     6.0  2096.0    4.0  0.0
3.0     66.0     1.0     8.0  499.0  0.0
4.0     10.0     0.0     0.0    4.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9812819822246163
Precision total:  0.7547839400757195
Recall total:  0.7635536935985494
F1 total:  0.7590703964537397
BACC total:  0.7635536935985494
MCC total:  0.9685429694278876


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


## bagging  with SVM


In [91]:
## bagging  with SVM
from sklearn.ensemble import BaggingClassifier
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score



from sklearn.linear_model import SGDClassifier

# Instantiate the SGDClassifier with additional hyperparameters
svm_01 = SGDClassifier(
    loss='hinge',           # hinge loss for linear SVM
    penalty='l2',           # L2 regularization to prevent overfitting
    alpha=1e-4,             # Learning rate (small value for fine-grained updates)
    max_iter=1000,          # Number of passes over the training data
    random_state=42,        # Seed for reproducible results
    learning_rate='optimal' # Automatically adjusts the learning rate based on the training data
)
# # Load the Iris dataset
# iris = load_iris()
# X, y = iris.data, iris.target

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define the base classifier (Decision Tree in this case)
base_classifier = svm_01

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train_01, y_train_01)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test_01)

# Evaluate accuracy
# accuracy = accuracy_score(y_test_01, y_pred)
# print(f'Accuracy: {accuracy}')

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

with open(output_file_name, "a") as f: print('Bagging with SVM', file = f)


print('---------------------------------------------------------------------------------')
print('CONFUSION MATRIX')
print('---------------------------------------------------------------------------------')


pred_label = y_pred

confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
all_unique_values = sorted(set(pred_label) | set(y_test_01))
z = np.zeros((len(all_unique_values), len(all_unique_values)))
rows, cols = confusion_matrix.shape
z[:rows, :cols] = confusion_matrix
confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
# confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
# with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
print(confusion_matrix)
with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)
TP_total = sum(TP)
TN_total = sum(TN)
FP_total = sum(FP)
FN_total = sum(FN)

TP_total = np.array(TP_total,dtype=np.float64)
TN_total = np.array(TN_total,dtype=np.float64)
FP_total = np.array(FP_total,dtype=np.float64)
FN_total = np.array(FN_total,dtype=np.float64)



#----------------------------------------------------------------#----------------------------------------------------------------

print('---------------------------------------------------------------------------------')
print('METRICS')
print('---------------------------------------------------------------------------------')


Acc = accuracy_score(y_test_01, pred_label)
Precision = precision_score(y_test_01, pred_label, average='macro')
Recall = recall_score(y_test_01, pred_label, average='macro')
F1 =  f1_score(y_test_01, pred_label, average='macro')
BACC = balanced_accuracy_score(y_test_01, pred_label)
MCC = matthews_corrcoef(y_test_01, pred_label)


bag_svm_acc_01 = Acc
bag_svm_pre_01 = Precision
bag_svm_rec_01 = Recall
bag_svm_f1_01 = F1
bag_svm_bacc_01 = BACC
bag_svm_mcc_01 = MCC
# with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
print('Accuracy total: ', Acc)
print('Precision total: ', Precision )
print('Recall total: ', Recall )
print('F1 total: ', F1 )
print('BACC total: ', BACC)
print('MCC total: ', MCC)

with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)




---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0    2.0  3.0  4.0
0.0  11094.0   223.0  111.0  0.0  0.0
1.0    522.0  7583.0    6.0  0.0  0.0
2.0     31.0  1950.0  170.0  0.0  0.0
3.0     63.0    29.0  482.0  0.0  0.0
4.0      8.0     2.0    4.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.8599964090133764
Precision total:  0.4690193067460854
Recall total:  0.5490796021769268
F1 total:  0.5043264490349578
BACC total:  0.5490796021769268
MCC total:  0.7656097087636203


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### DNN

In [46]:
print('---------------------------------------------------------------------------------')
print('Defining DNN Model')
print('---------------------------------------------------------------------------------')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#Model Parameters
dropout_rate = 0.2
nodes = 3
out_layer = 5
optimizer='adam'
loss='sparse_categorical_crossentropy'
epochs=100
batch_size=128


num_columns = X_train_01.shape[1]

dnn_01 = tf.keras.Sequential()

# Input layer
dnn_01.add(tf.keras.Input(shape=(num_columns,)))

# # Dense layers with dropout
# dnn_01.add(tf.keras.layers.Dense(nodes))
# dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

# dnn_01.add(tf.keras.layers.Dense(2*nodes))
# dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

# dnn_01.add(tf.keras.layers.Dense(3*nodes))
# dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

# dnn_01.add(tf.keras.layers.Dense(2*nodes))
# dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

# dnn.add(tf.keras.layers.Dense(nodes))
# dnn.add(tf.keras.layers.Dropout(dropout_rate))



# Dense layers with dropout
dnn_01.add(tf.keras.layers.Dense(nodes))
dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

dnn_01.add(tf.keras.layers.Dense(nodes))
dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

dnn_01.add(tf.keras.layers.Dense(nodes))
dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

dnn_01.add(tf.keras.layers.Dense(nodes))
dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

dnn_01.add(tf.keras.layers.Dense(nodes))
dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

# Output layer
# dnn_01.add(tf.keras.layers.Dense(out_layer))

dnn_01.add(tf.keras.layers.Dense(out_layer, activation='softmax'))


dnn_01.compile(optimizer=optimizer, loss=loss,metrics=['accuracy'])

dnn_01.summary()



---------------------------------------------------------------------------------
Defining DNN Model
---------------------------------------------------------------------------------
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 15        
_________________________________________________________________
dropout (Dropout)            (None, 3)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 3)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 12        
_________________________________________________________________
dropo

In [47]:
# X_train_02 = X_train_01

In [48]:
# X_train_02 = X_train_02.values

In [49]:
# X_train_03 = pd.DataFrame(X_train_02) 

In [50]:
# # Create a mask to filter out non-numeric values
# mask = np.char.isnumeric(X_train_02)

# # Apply the mask to the original array
# arr_numeric = np.where(mask, X_train_02, np.nan).astype(float)

# print(arr_numeric)

In [51]:
# X_train_02
# X_train_02 = X_train_02.astype(float)

# print(X_train_02)

In [52]:
# X_train_02 = X_train_01
# X_train_02.columns = None


In [53]:
#DNN
try:
    from keras.callbacks import EarlyStopping

    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

    print('---------------------------------------------------------------------------------')
    print('Training DNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training DNN', file = f)
    print('---------------------------------------------------------------------------------')
    # Convert Y_test back to its original format
    # y_test = np.argmax(Y_test, axis=1)

    # Start the timer
    start = time.time()
    # dnn_01.fit(X_train_01, y_train_01, epochs=epochs, batch_size=batch_size)
    dnn_01.fit(X_train_01, y_train_01, epochs=epochs, batch_size=batch_size,validation_split=0.2, callbacks=[early_stopping])

    # model.fit(x_train, Y_train, epochs=100, batch_size=128, validation_split=0.2, callbacks=[early_stopping])

    # End the timer
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    # joblib.dump(dnn_01, 'dnn_level_01.joblib')
    # dnn_01.save("dnn_level_01.h5")

    # Calculate the time taken and print it out
    # print(f'Time taken for training: {time_taken} seconds')
except: 
    None

---------------------------------------------------------------------------------
Training DNN
---------------------------------------------------------------------------------
Epoch 1/100
325/325 [==============================] - 1s 2ms/step - loss: 1.3347 - accuracy: 0.5873 - val_loss: 0.7099 - val_accuracy: 0.8708
Epoch 2/100
325/325 [==============================] - 1s 2ms/step - loss: 0.8427 - accuracy: 0.7290 - val_loss: 0.4766 - val_accuracy: 0.8578
Epoch 3/100
325/325 [==============================] - 1s 2ms/step - loss: 0.7015 - accuracy: 0.7833 - val_loss: 0.4180 - val_accuracy: 0.8572
Epoch 4/100
325/325 [==============================] - 1s 2ms/step - loss: 0.6381 - accuracy: 0.8062 - val_loss: 0.3873 - val_accuracy: 0.9292
Epoch 5/100
325/325 [==============================] - 1s 2ms/step - loss: 0.5971 - accuracy: 0.8254 - val_loss: 0.3836 - val_accuracy: 0.9291
Epoch 6/100
325/325 [==============================] - 1s 2ms/step - loss: 0.5660 - accuracy: 0.8408 - val_l

325/325 [==============================] - 1s 2ms/step - loss: 0.5571 - accuracy: 0.8475 - val_loss: 0.3642 - val_accuracy: 0.9362
Epoch 8/100
325/325 [==============================] - 1s 2ms/step - loss: 0.5390 - accuracy: 0.8539 - val_loss: 0.3567 - val_accuracy: 0.9417
Epoch 9/100
325/325 [==============================] - 1s 2ms/step - loss: 0.5327 - accuracy: 0.8571 - val_loss: 0.3563 - val_accuracy: 0.9351
Epoch 10/100
325/325 [==============================] - 1s 2ms/step - loss: 0.5210 - accuracy: 0.8618 - val_loss: 0.3533 - val_accuracy: 0.9351
Epoch 11/100
325/325 [==============================] - 1s 2ms/step - loss: 0.5167 - accuracy: 0.8663 - val_loss: 0.3527 - val_accuracy: 0.9344
Epoch 12/100
325/325 [==============================] - 1s 2ms/step - loss: 0.5102 - accuracy: 0.8726 - val_loss: 0.3509 - val_accuracy: 0.9426
Epoch 13/100
325/325 [==============================] - 1s 2ms/step - loss: 0.5099 - accuracy: 0.8761 - val_loss: 0.3492 - val_accuracy: 0.9414
Epoch 1

In [54]:
# dnn_01 = load_model("dnn_level_01.h5")


In [55]:
#DNN
try:
    start = time.time()
    pred_dnn = dnn_01.predict(X_test_01)
    preds_dnn_01 = np.argmax(pred_dnn,axis = 1)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
except:
        with open(output_file_name, "a") as f: print('error', file = f)
        preds_dnn_01 = 0


In [56]:
# y_test = y_test_01

In [57]:
#----------------------------------------------------------------
with open(output_file_name, "a") as f: print('Stack model - Strong learner - level 01', file = f)
with open(output_file_name, "a") as f: print('-------------------------------------------------------', file = f)

In [58]:
try:
    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')
    with open(output_file_name, "a") as f: print('DNN', file = f)
    pred_label = preds_dnn_01

    # pred_label = ypred
    #pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    dnn_acc_01 = Acc
    dnn_pre_01 = Precision
    dnn_rec_01 = Recall
    dnn_f1_01 = F1
    dnn_bacc_01 = BACC
    dnn_mcc_01 = MCC

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)
except: None


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0  3.0  4.0
0.0  11210.0    90.0   128.0  0.0  0.0
1.0     59.0  8041.0    11.0  0.0  0.0
2.0     45.0   348.0  1758.0  0.0  0.0
3.0     65.0     4.0   505.0  0.0  0.0
4.0     10.0     0.0     4.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9430379746835443
Precision total:  0.5325704376912934
Recall total:  0.5579176145445526
F1 total:  0.5446612807463953
BACC total:  0.5579176145445526
MCC total:  0.9041169056517298


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [59]:
#SVM
print('---------------------------------------------------------------------------------')
print('Defining SVM Model')
print('---------------------------------------------------------------------------------')

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier

# Instantiate the SGDClassifier with additional hyperparameters
clf = SGDClassifier(
    loss='hinge',           # hinge loss for linear SVM
    penalty='l2',           # L2 regularization to prevent overfitting
    alpha=1e-4,             # Learning rate (small value for fine-grained updates)
    max_iter=1000,          # Number of passes over the training data
    random_state=42,        # Seed for reproducible results
    learning_rate='optimal' # Automatically adjusts the learning rate based on the training data
)

#SVM
start = time.time()
clf.fit(X_train_01, y_train_01)
end = time.time()
clf.score(X_train_01, y_train_01)
time_taken = end - start
with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
joblib.dump(clf, 'svm_level_01.joblib')


clf = loaded_model = joblib.load('svm_level_01.joblib')


#SVM
start = time.time()
preds_svm_01 = clf.predict(X_test_01)
end = time.time()
time_taken = end - start
with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
print('---------------------------------------------------------------------------------')



---------------------------------------------------------------------------------
Defining SVM Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------


In [60]:
with open(output_file_name, "a") as f: print('-------------------------------------------------------', file = f)
print('---------------------------------------------------------------------------------')
print('CONFUSION MATRIX')
print('---------------------------------------------------------------------------------')
with open(output_file_name, "a") as f: print('SVM', file = f)
pred_label = preds_svm_01

# pred_label = ypred
#pred_label = label[ypred]

confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
all_unique_values = sorted(set(pred_label) | set(y_test_01))
z = np.zeros((len(all_unique_values), len(all_unique_values)))
rows, cols = confusion_matrix.shape
z[:rows, :cols] = confusion_matrix
confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
# confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
# with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
print(confusion_matrix)
with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)
TP_total = sum(TP)
TN_total = sum(TN)
FP_total = sum(FP)
FN_total = sum(FN)

TP_total = np.array(TP_total,dtype=np.float64)
TN_total = np.array(TN_total,dtype=np.float64)
FP_total = np.array(FP_total,dtype=np.float64)
FN_total = np.array(FN_total,dtype=np.float64)



#----------------------------------------------------------------#----------------------------------------------------------------

print('---------------------------------------------------------------------------------')
print('METRICS')
print('---------------------------------------------------------------------------------')

# Acc = ACC(TP_total,TN_total, FP_total, FN_total)
# Precision = PRECISION(TP_total, FP_total)
# Recall = RECALL(TP_total, FN_total)
# F1 = F1(Recall,Precision)
# BACC = BACC(TP_total,TN_total, FP_total, FN_total)
# MCC = MCC(TP_total,TN_total, FP_total, FN_total)


Acc = accuracy_score(y_test_01, pred_label)
Precision = precision_score(y_test_01, pred_label, average='macro')
Recall = recall_score(y_test_01, pred_label, average='macro')
F1 =  f1_score(y_test_01, pred_label, average='macro')
BACC = balanced_accuracy_score(y_test_01, pred_label)
MCC = matthews_corrcoef(y_test_01, pred_label)



svm_acc_01 = Acc
svm_pre_01 = Precision
svm_rec_01 = Recall
svm_f1_01 = F1
svm_bacc_01 = BACC
svm_mcc_01 = MCC

# with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
print('Accuracy total: ', Acc)
print('Precision total: ', Precision )
print('Recall total: ', Recall )
print('F1 total: ', F1 )
print('BACC total: ', BACC)
print('MCC total: ', MCC)

with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0  2.0    3.0  4.0
0.0  11056.0   261.0  0.0  111.0  0.0
1.0     57.0  8048.0  0.0    6.0  0.0
2.0     30.0  1950.0  1.0  170.0  0.0
3.0     63.0    28.0  1.0  482.0  0.0
4.0      8.0     2.0  0.0    4.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.8792081874495018
Precision total:  0.5783299354191134
Recall total:  0.5599734594264867
F1 total:  0.5135936316789838
BACC total:  0.5599734594264867
MCC total:  0.8036574573652664


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [61]:

print('---------------------------------------------------------------------------------')
print('Defining RF Model')
print('---------------------------------------------------------------------------------')
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
rf = RandomForestClassifier(max_depth = 5,  n_estimators = 10, min_samples_split = 2, n_jobs = -1)
#------------------------------------------------------------------------------

if True == True:

    print('---------------------------------------------------------------------------------')
    print('Training RF')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)
    with open(output_file_name, "a") as f: print('Training RF', file = f)
    print('---------------------------------------------------------------------------------')
    #RF
    start = time.time()
    model_rf_01 = rf.fit(X_train_01,y_train_01)
    end = time.time()

    # # Create the StratifiedKFold object
    # stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # # Perform cross-validation
    # cv_scores = cross_val_score(model_rf_01, X_train_01, y_train, cv=stratified_kfold, scoring='accuracy')
    # # Print the cross-validation scores
    # print("Cross-validation scores:", cv_scores)
    # print("Mean accuracy:", cv_scores.mean())
    # with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(model_rf_01, 'rf_base_model_01.joblib')

if 1 == 1:
    model_rf_01  = joblib.load('rf_base_model_01.joblib')

if 1 == 1:

    print('---------------------------------------------------------------------------------')
    print('Prediction RF')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction RF', file = f)
    print('---------------------------------------------------------------------------------')
    #RF
    start = time.time()
    preds_rf_01 = model_rf_01.predict(X_test_01)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

    with open(output_file_name, "a") as f: print('-------------------------------------------------------', file = f)
print('---------------------------------------------------------------------------------')
print('CONFUSION MATRIX')
print('---------------------------------------------------------------------------------')
with open(output_file_name, "a") as f: print('RF', file = f)
pred_label = preds_rf_01

# pred_label = ypred
#pred_label = label[ypred]

confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
all_unique_values = sorted(set(pred_label) | set(y_test_01))
z = np.zeros((len(all_unique_values), len(all_unique_values)))
rows, cols = confusion_matrix.shape
z[:rows, :cols] = confusion_matrix
confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
# confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
# with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
print(confusion_matrix)
with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)
TP_total = sum(TP)
TN_total = sum(TN)
FP_total = sum(FP)
FN_total = sum(FN)

TP_total = np.array(TP_total,dtype=np.float64)
TN_total = np.array(TN_total,dtype=np.float64)
FP_total = np.array(FP_total,dtype=np.float64)
FN_total = np.array(FN_total,dtype=np.float64)



#----------------------------------------------------------------#----------------------------------------------------------------

print('---------------------------------------------------------------------------------')
print('METRICS')
print('---------------------------------------------------------------------------------')

# Acc = ACC(TP_total,TN_total, FP_total, FN_total)
# Precision = PRECISION(TP_total, FP_total)
# Recall = RECALL(TP_total, FN_total)
# F1 = F1(Recall,Precision)
# BACC = BACC(TP_total,TN_total, FP_total, FN_total)
# MCC = MCC(TP_total,TN_total, FP_total, FN_total)


Acc = accuracy_score(y_test_01, pred_label)
Precision = precision_score(y_test_01, pred_label, average='macro')
Recall = recall_score(y_test_01, pred_label, average='macro')
F1 =  f1_score(y_test_01, pred_label, average='macro')
BACC = balanced_accuracy_score(y_test_01, pred_label)
MCC = matthews_corrcoef(y_test_01, pred_label)

rf_acc_01 = Acc
rf_pre_01 = Precision
rf_rec_01 = Recall
rf_f1_01 = F1
rf_bacc_01 = BACC
rf_mcc_01 = MCC


# with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
print('Accuracy total: ', Acc)
print('Precision total: ', Precision )
print('Recall total: ', Recall )
print('F1 total: ', F1 )
print('BACC total: ', BACC)
print('MCC total: ', MCC)

with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)




---------------------------------------------------------------------------------
Defining RF Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training RF
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction RF
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0    3.0  4.0
0.0  11210.0    70.0    54.0   94.0  0.0
1.0     48.0  8050.0     8.0    5.0  0.0
2.0     45.0     5.0  2097.0    4.0  0.0
3.0     64.0     1.0     8.0  501.0  

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [62]:
rf_acc_01

0.9811473202262322

In [63]:
print('---------------------------------------------------------------------------------')
print('Defining LGBM Model')
print('---------------------------------------------------------------------------------')
#LGBM
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()



if 1 == 1 and 0 == 0:


    print('---------------------------------------------------------------------------------')
    print('Training LGBM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training LGBM', file = f)
    print('---------------------------------------------------------------------------------')
    start = time.time()
    lgbm.fit(X_train_01, y_train_01)
    end = time.time()

    # # Create the StratifiedKFold object
    # stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # # Perform cross-validation
    # cv_scores = cross_val_score(lgbm, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # # Print the cross-validation scores
    # print("Cross-validation scores:", cv_scores)
    # print("Mean accuracy:", cv_scores.mean())
    # with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)

    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(lgbm, 'lgbm_01.joblib')

if 1 == 1:
    lgbm = joblib.load('lgbm_01.joblib')


if 1 == 1:

    print('Prediction LGBM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction LGBM', file = f)
    print('---------------------------------------------------------------------------------')
    #LGBM
    start = time.time()
    preds_lgbm_01 = lgbm.predict(X_test_01)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')
    with open(output_file_name, "a") as f: print('LGBM', file = f)
    pred_label = preds_lgbm_01

    # pred_label = ypred
    #pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)


    Acc = accuracy_score(y_test_01, pred_label)
    lgbm_acc_01 = Acc
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)

    lgbm_acc_01 = Acc
    lgbm_pre_01 = Precision
    lgbm_rec_01 = Recall
    lgbm_f1_01 = F1
    lgbm_bacc_01 = BACC
    lgbm_mcc_01 = MCC

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)





---------------------------------------------------------------------------------
Defining LGBM Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training LGBM
---------------------------------------------------------------------------------


Prediction LGBM
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0    3.0  4.0
0.0  11213.0    70.0    54.0   91.0  0.0
1.0     50.0  8053.0     4.0    4.0  0.0
2.0     45.0     6.0  2096.0    4.0  0.0
3.0     66.0     1.0     8.0  499.0  0.0
4.0     10.0     0.0     0.0    4.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9812819822246163
Precision total:  0.7547768376013453
Recall total:  0.7635608505955311
F1 total:  0.7590703045266477
BACC total:  0.7635608505955311
MCC total:  0.9685440983622206


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [64]:
# lgbm_acc_01

In [65]:

#MLP
print('---------------------------------------------------------------------------------')
print('Defining MLP Model')
print('---------------------------------------------------------------------------------')


from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
import time

# create MLPClassifier instance
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=1)

if 1 == 1 and 0 == 0:


    print('---------------------------------------------------------------------------------')
    print('Training MLP')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training MLP', file = f)
    print('---------------------------------------------------------------------------------')

    start = time.time()
    MLP = mlp.fit(X_train_01, y_train_01)
    end = time.time()

    # # Create the StratifiedKFold object
    # stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # # Perform cross-validation
    # cv_scores = cross_val_score(MLP, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # # Print the cross-validation scores
    # print("Cross-validation scores:", cv_scores)
    # print("Mean accuracy:", cv_scores.mean())
    # with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)

    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(MLP, 'mlp_01.joblib')

if 1 == 1:
    MLP = joblib.load('mlp_01.joblib')


if 1 == 1:

    #MLP
    start = time.time()
    y_pred = MLP.predict_proba(X_test_01)
    preds_mlp_01 = np.argmax(y_pred,axis = 1)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#MLP
if 1 == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('MLP 01 model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_mlp_01
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    mlp_acc_01 = Acc
    mlp_pre_01 = Precision
    mlp_rec_01 = Recall
    mlp_f1_01 = F1
    mlp_bacc_01 = BACC
    mlp_mcc_01 = MCC

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)








---------------------------------------------------------------------------------
Defining MLP Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training MLP
---------------------------------------------------------------------------------


---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0    3.0  4.0
0.0  11210.0    70.0    55.0   93.0  0.0
1.0     49.0  8053.0     4.0    5.0  0.0
2.0     46.0     5.0  2097.0    3.0  0.0
3.0     66.0     1.0     9.0  498.0  0.0
4.0     10.0     0.0     0.0    4.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9811473202262322
Precision total:  0.7540175171198078
Recall total:  0.7632528959239488
F1 total:  0.7585292769243145
BACC total:  0.7632528959239488
MCC total:  0.9683222763644737


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [66]:
# mlp_acc_01

In [67]:
print('---------------------------------------------------------------------------------')
print('Defining ADA Model')
print('---------------------------------------------------------------------------------')
#ADA
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
import time
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)

if 1 == 1 and 0 == 0:

    print('---------------------------------------------------------------------------------')
    print('Training ADA')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training ADA', file = f)
    print('---------------------------------------------------------------------------------')
    #ADA


    start = time.time()
    ada = abc.fit(X_train_01, y_train_01)
    end = time.time()

    # # Create the StratifiedKFold object
    # stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # # Perform cross-validation
    # cv_scores = cross_val_score(ada, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # # Print the cross-validation scores
    # print("Cross-validation scores:", cv_scores)
    # print("Mean accuracy:", cv_scores.mean())
    # with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)

    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)

    # Assuming 'model' is your trained model
    joblib.dump(ada, 'ada_01.joblib')




if 1 == 1:
    ada = joblib.load('ada_01.joblib')


if 1 == 1:

    print('Prediction ADA')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction ADA', file = f)
    print('---------------------------------------------------------------------------------')
    #ADA
    start = time.time()
    preds_ada_01 = ada.predict(X_test_01)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if 1 == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('ADA 01 model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_ada_01
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)

    ada_acc_01 = Acc
    ada_pre_01 = Precision
    ada_rec_01 = Recall
    ada_f1_01 = F1
    ada_bacc_01 = BACC
    ada_mcc_01 = MCC

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)









---------------------------------------------------------------------------------
Defining ADA Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training ADA
---------------------------------------------------------------------------------


Prediction ADA
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0    3.0  4.0
0.0  11246.0    70.0    43.0   69.0  0.0
1.0     51.0  8017.0    39.0    4.0  0.0
2.0     78.0   189.0  1880.0    4.0  0.0
3.0    171.0     1.0     7.0  395.0  0.0
4.0     10.0     0.0     0.0    4.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9667833737319328
Precision total:  0.7452786169091856
Recall total:  0.7069300802727743
F1 total:  0.724398988880189
BACC total:  0.7069300802727743
MCC total:  0.943830575668782


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [68]:
#KNN
print('---------------------------------------------------------------------------------')
print('Defining KNN Model')
print('---------------------------------------------------------------------------------')
from sklearn.neighbors import KNeighborsClassifier
knn_clf_01=KNeighborsClassifier(n_neighbors = 5)

if 1 == 1 and 0 == 0:

    #KNN
    print('---------------------------------------------------------------------------------')
    print('Training KNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training KNN', file = f)
    print('---------------------------------------------------------------------------------')
    start = time.time()
    knn_clf_01.fit(X_train_01,y_train_01)
    end = time.time()


    # # Create the StratifiedKFold object
    # stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # # Perform cross-validation
    # cv_scores = cross_val_score(knn_clf, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # # Print the cross-validation scores
    # print("Cross-validation scores:", cv_scores)
    # print("Mean accuracy:", cv_scores.mean())
    # with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(knn_clf_01, 'knn_01.joblib')


if load_model_knn == 1:
    knn_clf_01 = joblib.load('knn_01.joblib')

if use_model_knn == 1:

    #KNN
    start = time.time()
    preds_knn =knn_clf_01.predict(X_test_01)
    preds_knn
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#MLP
if 1 == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('KNN 01 model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_knn
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    knn_acc_01 = Acc
    knn_pre_01 = Precision
    knn_rec_01 = Recall
    knn_f1_01 = F1
    knn_bacc_01 = BACC
    knn_mcc_01 = MCC    

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)









---------------------------------------------------------------------------------
Defining KNN Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training KNN
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0    3.0  4.0
0.0  11225.0    71.0    44.0   88.0  0.0
1.0     55.0  8048.0     4.0    4.0  0.0
2.0     77.0     6.0  2065.0    3.0  0.0
3.0     70.0     1.0    11.0  492.0  0.0
4.0     10.0     0.0     0.0    4.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.979890474907981
Precis

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [69]:
from sklearn.linear_model import LogisticRegression

#Logistic Regression
print('---------------------------------------------------------------------------------')
print('Defining Logistic Regression Model')
print('---------------------------------------------------------------------------------')
logreg_01 = LogisticRegression()

if 1 == 1 and 0 == 0:

    #KNN
    print('---------------------------------------------------------------------------------')
    print('Training LR ')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training LR', file = f)
    print('---------------------------------------------------------------------------------')
    start = time.time()
    logreg_01.fit(X_train_01,y_train_01)
    end = time.time()


    # # Create the StratifiedKFold object
    # stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # # Perform cross-validation
    # cv_scores = cross_val_score(knn_clf, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # # Print the cross-validation scores
    # print("Cross-validation scores:", cv_scores)
    # print("Mean accuracy:", cv_scores.mean())
    # with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(logreg_01, 'logreg_01.joblib')


if 1 == 1:
    logreg_01 = joblib.load('logreg_01.joblib')

if 1 == 1:

    #lR
    start = time.time()
    preds_logreg =logreg_01.predict(X_test_01)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#LR
if 1 == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('LR 01 model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_logreg
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)

    lr_acc_01 = Acc
    lr_pre_01 = Precision
    lr_rec_01 = Recall
    lr_f1_01 = F1
    lr_bacc_01 = BACC
    lr_mcc_01 = MCC

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)









---------------------------------------------------------------------------------
Defining Logistic Regression Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training LR 
---------------------------------------------------------------------------------


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0    3.0  4.0
0.0  11073.0   224.0    37.0   94.0  0.0
1.0     46.0  8054.0     7.0    4.0  0.0
2.0     30.0    96.0  1907.0  118.0  0.0
3.0     62.0     4.0    27.0  481.0  0.0
4.0      8.0     2.0     0.0    4.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9657509650776551
Precision total:  0.7196703991990359
Recall total:  0.737290387200625
F1 total:  0.7265746441494881
BACC total:  0.737290387200625
MCC total:  0.9427310738311581


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [70]:
# #Voting
# from sklearn.ensemble import VotingClassifier
# # model1 = LogisticRegression(random_state=1)
# # model2 = tree.DecisionTreeClassifier(random_state=1)
# voting = VotingClassifier(estimators=[
#                                         ('ada', ada),
#                                        ('rf', rf),
#                                        ('svm', clf),
#                                        ('knn', knn_clf), 
#                                        ('lgbm', lgbm),
#                                       #  ('xgb', xgb_00),
#                                        ('cat', cat_00),

#                                          ('mlp', mlp)
#                                         #  ,('dnn', dnn_01)

#                                          ], voting='hard')
# voting.fit(X_train_01,y_train_01)
# # voring_acc = voting.score(X_test_01,y_test_01)

In [71]:
# preds_voting = voting(X_test_01,y_test_01)

In [72]:

# print('---------------------------------------------------------------------------------')
# print('CONFUSION MATRIX')
# print('---------------------------------------------------------------------------------')


# pred_label = preds_voting
# # pred_label = label[ypred]

# confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
# all_unique_values = sorted(set(pred_label) | set(y_test_01))
# z = np.zeros((len(all_unique_values), len(all_unique_values)))
# rows, cols = confusion_matrix.shape
# z[:rows, :cols] = confusion_matrix
# confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
# # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
# # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
# print(confusion_matrix)
# with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

# with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


# FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
# FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
# TP = np.diag(confusion_matrix)
# TN = confusion_matrix.values.sum() - (FP + FN + TP)
# TP_total = sum(TP)
# TN_total = sum(TN)
# FP_total = sum(FP)
# FN_total = sum(FN)

# TP_total = np.array(TP_total,dtype=np.float64)
# TN_total = np.array(TN_total,dtype=np.float64)
# FP_total = np.array(FP_total,dtype=np.float64)
# FN_total = np.array(FN_total,dtype=np.float64)



# #----------------------------------------------------------------#----------------------------------------------------------------

# print('---------------------------------------------------------------------------------')
# print('METRICS')
# print('---------------------------------------------------------------------------------')

# # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
# # Precision = PRECISION(TP_total, FP_total)
# # Recall = RECALL(TP_total, FN_total)
# # F1 = F1(Recall,Precision)
# # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
# # MCC = MCC(TP_total,TN_total, FP_total, FN_total)


# Acc = accuracy_score(y_test_01, pred_label)
# Precision = precision_score(y_test_01, pred_label, average='macro')
# Recall = recall_score(y_test_01, pred_label, average='macro')
# F1 =  f1_score(y_test_01, pred_label, average='macro')
# BACC = balanced_accuracy_score(y_test_01, pred_label)
# MCC = matthews_corrcoef(y_test_01, pred_label)


# voting_acc_01 = Acc
# voting_pre_01 = Precision
# voting_rec_01 = Recall
# voting_f1_01 = F1
# voting_bacc_01 = BACC
# voting_mcc_01 = MCC

# # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
# print('Accuracy total: ', Acc)
# print('Precision total: ', Precision )
# print('Recall total: ', Recall )
# print('F1 total: ', F1 )
# print('BACC total: ', BACC)
# print('MCC total: ', MCC)

# with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
# with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
# with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
# with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
# with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
# with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)









In [73]:
# from sklearn.calibration import CalibratedClassifierCV
# with open(output_file_name, "a") as f: print('Generating Predictions', file = f)

# if use_model_rf == 1:

#     print('---------------------------------------------------------------------------------')
#     print('Prediction RF')
#     with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#     with open(output_file_name, "a") as f: print('Prediction RF', file = f)
#     print('---------------------------------------------------------------------------------')
#     #RF
#     start = time.time()
#     preds_rf = rf.predict(X_test)
#     preds_rf_prob = rf.predict_proba(X_test)
#     end = time.time()
#     time_taken = end - start
#     with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
#     print('---------------------------------------------------------------------------------')

# if use_model_svm == 1:

#     print('Prediction SVM')
#     with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#     with open(output_file_name, "a") as f: print('Prediction SVM', file = f)
#     print('---------------------------------------------------------------------------------')
#     #SVM
#     start = time.time()
#     preds_svm = clf.predict(X_test)
#     # preds_svm_prob = clf.predict_proba(X_test)

#     #Since SVM does not deal with prob by nature we use a meta learner
#     # https://stackoverflow.com/questions/55250963/how-to-get-probabilities-for-sgdclassifier-linearsvm

#     model = CalibratedClassifierCV(clf)

#     model.fit(X, y)
#     preds_svm_prob = model.predict_proba(X)

#     end = time.time()
#     time_taken = end - start
#     with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
#     print('---------------------------------------------------------------------------------')

# if use_model_lgbm == 1:

#     print('Prediction LGBM')
#     with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#     with open(output_file_name, "a") as f: print('Prediction LGBM', file = f)
#     print('---------------------------------------------------------------------------------')
#     #LGBM
#     start = time.time()
#     preds_lgbm = lgbm.predict(X_test)
#     preds_lgbm_prob = lgbm.predict_proba(X_test)

#     end = time.time()
#     time_taken = end - start
#     with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
#     print('---------------------------------------------------------------------------------')

# if use_model_dnn == 1:

#     print('Prediction DNN')
#     with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#     with open(output_file_name, "a") as f: print('Prediction DNN', file = f)
#     print('---------------------------------------------------------------------------------')
#     #DNN
#     start = time.time()
#     pred_dnn = dnn.predict(X_test)
#     preds_dnn_prob = pred_dnn
#     preds_dnn = np.argmax(pred_dnn,axis = 1)
#     end = time.time()
#     time_taken = end - start
#     with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
#     print('---------------------------------------------------------------------------------')

# if use_model_ada == 1:

#     print('Prediction ADA')
#     with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#     with open(output_file_name, "a") as f: print('Prediction ADA', file = f)
#     print('---------------------------------------------------------------------------------')
#     #ADA
#     start = time.time()
#     preds_ada = ada.predict(X_test)
#     preds_ada_prob = ada.predict_proba(X_test)

#     end = time.time()
#     time_taken = end - start
#     with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
#     print('---------------------------------------------------------------------------------')
#     print('Prediction MLP')
#     with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#     with open(output_file_name, "a") as f: print('Prediction MLP', file = f)
#     print('---------------------------------------------------------------------------------')

# if use_model_mlp == 1:

#     #MLP
#     start = time.time()
#     y_pred = MLP.predict_proba(X_test)
#     preds_mlp_prob = y_pred
#     preds_mlp = np.argmax(y_pred,axis = 1)
#     end = time.time()
#     time_taken = end - start
#     with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
#     print('---------------------------------------------------------------------------------')
#     print('Prediction KNN')
#     with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#     with open(output_file_name, "a") as f: print('Prediction KNN', file = f)
#     print('---------------------------------------------------------------------------------')

# if use_model_knn == 1:

#     #KNN
#     start = time.time()
#     preds_knn =knn_clf.predict(X_test)
#     preds_knn_prob =knn_clf.predict_proba(X_test)

#     preds_knn
#     end = time.time()
#     time_taken = end - start
#     with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
#     with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)


In [74]:
import catboost

cat_01 = catboost.CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass', custom_metric='Accuracy')

# Fit the model
cat_01.fit(X_train_01, y_train_01, eval_set=(X_test_01, y_test_01), verbose=10)

# Make predictions on the test set
preds_cat = cat_01.predict(X_test_01)
preds_cat = np.squeeze(preds_cat)

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

with open(output_file_name, "a") as f: print('catboost', file = f)


print('---------------------------------------------------------------------------------')
print('CONFUSION MATRIX')
print('---------------------------------------------------------------------------------')


# pred_label = label[ypred]

confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
all_unique_values = sorted(set(pred_label) | set(y_test_01))
z = np.zeros((len(all_unique_values), len(all_unique_values)))
rows, cols = confusion_matrix.shape
z[:rows, :cols] = confusion_matrix
confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
# confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
# with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
print(confusion_matrix)
with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)
TP_total = sum(TP)
TN_total = sum(TN)
FP_total = sum(FP)
FN_total = sum(FN)

TP_total = np.array(TP_total,dtype=np.float64)
TN_total = np.array(TN_total,dtype=np.float64)
FP_total = np.array(FP_total,dtype=np.float64)
FN_total = np.array(FN_total,dtype=np.float64)



#----------------------------------------------------------------#----------------------------------------------------------------

print('---------------------------------------------------------------------------------')
print('METRICS')
print('---------------------------------------------------------------------------------')


Acc = accuracy_score(y_test_01, pred_label)
Precision = precision_score(y_test_01, pred_label, average='macro')
Recall = recall_score(y_test_01, pred_label, average='macro')
F1 =  f1_score(y_test_01, pred_label, average='macro')
BACC = balanced_accuracy_score(y_test_01, pred_label)
MCC = matthews_corrcoef(y_test_01, pred_label)


cat_acc_01 = Acc
cat_pre_01 = Precision
cat_rec_01 = Recall
cat_f1_01 = F1
cat_bacc_01 = BACC
cat_mcc_01 = MCC

# with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
print('Accuracy total: ', Acc)
cat_acc_01 = Acc
print('Precision total: ', Precision )
print('Recall total: ', Recall )
print('F1 total: ', F1 )
print('BACC total: ', BACC)
print('MCC total: ', MCC)

with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)



0:	learn: 1.2495200	test: 1.2489695	best: 1.2489695 (0)	total: 59.5ms	remaining: 5.89s
10:	learn: 0.3443327	test: 0.3418232	best: 0.3418232 (10)	total: 131ms	remaining: 1.06s
20:	learn: 0.1691488	test: 0.1659377	best: 0.1659377 (20)	total: 177ms	remaining: 664ms
30:	learn: 0.1156980	test: 0.1124291	best: 0.1124291 (30)	total: 221ms	remaining: 492ms


40:	learn: 0.0980574	test: 0.0947730	best: 0.0947730 (40)	total: 265ms	remaining: 381ms
50:	learn: 0.0920404	test: 0.0890359	best: 0.0890359 (50)	total: 310ms	remaining: 298ms
60:	learn: 0.0898443	test: 0.0871308	best: 0.0871308 (60)	total: 353ms	remaining: 226ms
70:	learn: 0.0888353	test: 0.0863192	best: 0.0863192 (70)	total: 395ms	remaining: 162ms
80:	learn: 0.0882694	test: 0.0859731	best: 0.0859731 (80)	total: 438ms	remaining: 103ms
90:	learn: 0.0878974	test: 0.0858086	best: 0.0858086 (90)	total: 482ms	remaining: 47.6ms
99:	learn: 0.0876242	test: 0.0856751	best: 0.0856672 (97)	total: 521ms	remaining: 0us

bestTest = 0.08566721955
bestIteration = 97

Shrink model to first 98 iterations.
---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0    3.0  4.0
0.0  11073.0   224.0    37.0   94.0  0.0
1.0     46.0  8054.0     7.0    4.0  0.

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [75]:

import xgboost as xgb

# Create a DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train_01, label=y_train_01)
dtest = xgb.DMatrix(X_test_01, label=y_test_01)

# Set XGBoost parameters
params = {
    'objective': 'multi:softmax',  # for multi-class classification
    'num_class': 5,  # specify the number of classes
    'max_depth': 3,
    'learning_rate': 0.1,
    'eval_metric': 'mlogloss'  # metric for multi-class classification
}

# Train the XGBoost model
num_round = 100
xgb_01 = xgb.train(params, dtrain, num_round)

# Make predictions on the test set
preds_xgb_01 = xgb_01.predict(dtest)


if 1 == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('xgboost base model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_xgb_01
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)

    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)

    xgb_acc_01 = Acc
    xgb_pre_01 = Precision
    xgb_rec_01 = Recall
    xgb_f1_01 = F1
    xgb_bacc_01 = BACC
    xgb_mcc_01 = MCC


    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0     1.0     2.0    3.0  4.0
0.0  11213.0    70.0    54.0   91.0  0.0
1.0     49.0  8054.0     4.0    4.0  0.0
2.0     45.0     5.0  2097.0    4.0  0.0
3.0     66.0     1.0     8.0  499.0  0.0
4.0     10.0     0.0     0.0    4.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9813717568902056
Precision total:  0.7548215666671336
Recall total:  0.7636784884768548
F1 total:  0.7591512142163392
BACC total:  0.7636784884768548
MCC total:  0.968696195385652


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [76]:
# model1 = tree.DecisionTreeClassifier()
# model2 = KNeighborsClassifier()
# model3= LogisticRegression()

# model1.fit(x_train,y_train)
# model2.fit(x_train,y_train)
# model3.fit(x_train,y_train)

# pred1=model1.predict_proba(x_test)
# pred2=model2.predict_proba(x_test)
# pred3=model3.predict_proba(x_test)

# finalpred=(preds_svm_prob +
#             preds_ada_prob +
#             preds_knn_prob +
#             preds_rf_prob +
#             preds_dnn_prob +
#             preds_lgbm_prob +
#             preds_mlp_prob
#             )/7

In [77]:
# with open(output_file_name, "a") as f: print('-----------------------', file = f)
# with open(output_file_name, "a") as f: print('Summary', file = f)
# with open(output_file_name, "a") as f: print('-----------------------', file = f)
# with open(output_file_name, "a") as f: print('Level 00', file = f)

# with open(output_file_name, "a") as f: print('Accuracy ada: ', ada_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy dnn: ', dnn_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy svm: ', svm_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy knn: ', knn_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy mlp: ', mlp_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy lgbm: ', lgbm_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy rf: ', rf_acc_00, file = f)

# with open(output_file_name, "a") as f: print('-----------------------', file = f)
# with open(output_file_name, "a") as f: print('Level 01', file = f)

# with open(output_file_name, "a") as f: print('Accuracy ada: ', ada_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy dnn: ', dnn_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy svm: ', svm_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy knn: ', knn_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy mlp: ', mlp_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy lgbm: ', lgbm_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy rf: ', rf_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy LR: ', lr_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy Voting: ', voring_acc, file = f)
# with open(output_file_name, "a") as f: print('Accuracy catboost: ', cat_acc_01, file = f)

In [78]:
# with open(output_file_name, "a") as f: print('-----------------------', file = f)
# with open(output_file_name, "a") as f: print('Summary', file = f)
# with open(output_file_name, "a") as f: print('-----------------------', file = f)
# with open(output_file_name, "a") as f: print('Level 00', file = f)

# with open(output_file_name, "a") as f: print('Accuracy ada: ', ada_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy dnn: ', dnn_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy svm: ', svm_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy knn: ', knn_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy mlp: ', mlp_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy lgbm: ', lgbm_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy rf: ', rf_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy cat: ', cat_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy xgb: ', xgb_acc_00, file = f)


# with open(output_file_name, "a") as f: print('Precision ada: ', ada_pre_00, file = f)
# with open(output_file_name, "a") as f: print('Precision dnn: ', dnn_pre_00, file = f)
# with open(output_file_name, "a") as f: print('Precision svm: ', svm_pre_00, file = f)
# with open(output_file_name, "a") as f: print('Precision knn: ', knn_pre_00, file = f)
# with open(output_file_name, "a") as f: print('Precision mlp: ', mlp_pre_00, file = f)
# with open(output_file_name, "a") as f: print('Precision lgbm: ', lgbm_pre_00, file = f)
# with open(output_file_name, "a") as f: print('Precision rf: ', rf_pre_00, file = f)
# with open(output_file_name, "a") as f: print('Precision cat: ', cat_pre_00, file = f)
# with open(output_file_name, "a") as f: print('Precision xgb: ', xgb_pre_00, file = f)

# with open(output_file_name, "a") as f: print('Recall ada: ', ada_rec_00, file = f)
# with open(output_file_name, "a") as f: print('Recall dnn: ', dnn_rec_00, file = f)
# with open(output_file_name, "a") as f: print('Recall svm: ', svm_rec_00, file = f)
# with open(output_file_name, "a") as f: print('Recall knn: ', knn_rec_00, file = f)
# with open(output_file_name, "a") as f: print('Recall mlp: ', mlp_rec_00, file = f)
# with open(output_file_name, "a") as f: print('Recall lgbm: ', lgbm_rec_00, file = f)
# with open(output_file_name, "a") as f: print('Recall rf: ', rf_rec_00, file = f)
# with open(output_file_name, "a") as f: print('Recall cat: ', cat_rec_00, file = f)
# with open(output_file_name, "a") as f: print('Recall xgb: ', xgb_rec_00, file = f)

# with open(output_file_name, "a") as f: print('F1 ada: ', ada_f1_00, file = f)
# with open(output_file_name, "a") as f: print('F1 dnn: ', dnn_f1_00, file = f)
# with open(output_file_name, "a") as f: print('F1 svm: ', svm_f1_00, file = f)
# with open(output_file_name, "a") as f: print('F1 knn: ', knn_f1_00, file = f)
# with open(output_file_name, "a") as f: print('F1 mlp: ', mlp_f1_00, file = f)
# with open(output_file_name, "a") as f: print('F1 lgbm: ', lgbm_f1_00, file = f)
# with open(output_file_name, "a") as f: print('F1 rf: ', rf_f1_00, file = f)
# with open(output_file_name, "a") as f: print('F1 cat: ', cat_f1_00, file = f)
# with open(output_file_name, "a") as f: print('F1 xgb: ', xgb_f1_00, file = f)


# with open(output_file_name, "a") as f: print('-----------------------', file = f)
# with open(output_file_name, "a") as f: print('Level 01', file = f)

# with open(output_file_name, "a") as f: print('Accuracy ada: ', ada_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy dnn: ', dnn_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy svm: ', svm_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy knn: ', knn_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy mlp: ', mlp_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy lgbm: ', lgbm_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy rf: ', rf_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy LR: ', lr_acc_01, file = f)
# # with open(output_file_name, "a") as f: print('Accuracy Voting: ', voting_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy catboost: ', cat_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy xgb: ', xgb_acc_01, file = f)

# with open(output_file_name, "a") as f: print('Precision ada: ', ada_pre_01, file = f)
# with open(output_file_name, "a") as f: print('Precision dnn: ', dnn_pre_01, file = f)
# with open(output_file_name, "a") as f: print('Precision svm: ', svm_pre_01, file = f)
# with open(output_file_name, "a") as f: print('Precision knn: ', knn_pre_01, file = f)
# with open(output_file_name, "a") as f: print('Precision mlp: ', mlp_pre_01, file = f)
# with open(output_file_name, "a") as f: print('Precision lgbm: ', lgbm_pre_01, file = f)
# with open(output_file_name, "a") as f: print('Precision rf: ', rf_pre_01, file = f)
# with open(output_file_name, "a") as f: print('Precision LR: ', lr_pre_01, file = f)
# # with open(output_file_name, "a") as f: print('Precision Voting: ', voting_pre_01, file = f)
# with open(output_file_name, "a") as f: print('Precision catboosting: ', cat_pre_01, file = f)
# with open(output_file_name, "a") as f: print('Precision xgboost: ', xgb_pre_01, file = f)

# with open(output_file_name, "a") as f: print('Recall ada: ', ada_rec_01, file = f)
# with open(output_file_name, "a") as f: print('Recall dnn: ', dnn_rec_01, file = f)
# with open(output_file_name, "a") as f: print('Recall svm: ', svm_rec_01, file = f)
# with open(output_file_name, "a") as f: print('Recall knn: ', knn_rec_01, file = f)
# with open(output_file_name, "a") as f: print('Recall mlp: ', mlp_rec_01, file = f)
# with open(output_file_name, "a") as f: print('Recall lgbm: ', lgbm_rec_01, file = f)
# with open(output_file_name, "a") as f: print('Recall rf: ', rf_rec_01, file = f)
# with open(output_file_name, "a") as f: print('Recall LR: ', lr_rec_01, file = f)
# # with open(output_file_name, "a") as f: print('Recall Voting: ', voting_rec_01, file = f)
# with open(output_file_name, "a") as f: print('Recall catboosting: ', cat_rec_01, file = f)
# with open(output_file_name, "a") as f: print('Recall xgboost: ', xgb_rec_01, file = f)

# with open(output_file_name, "a") as f: print('F1 ada: ', ada_f1_01, file = f)
# with open(output_file_name, "a") as f: print('F1 dnn: ', dnn_f1_01, file = f)
# with open(output_file_name, "a") as f: print('F1 svm: ', svm_f1_01, file = f)
# with open(output_file_name, "a") as f: print('F1 knn: ', knn_f1_01, file = f)
# with open(output_file_name, "a") as f: print('F1 mlp: ', mlp_f1_01, file = f)
# with open(output_file_name, "a") as f: print('F1 lgbm: ', lgbm_f1_01, file = f)
# with open(output_file_name, "a") as f: print('F1 rf: ', rf_f1_01, file = f)
# with open(output_file_name, "a") as f: print('F1 LR: ', lr_f1_01, file = f)
# # with open(output_file_name, "a") as f: print('F1 Voting: ', voting_f1_01, file = f)
# with open(output_file_name, "a") as f: print('F1 catboosting: ', cat_f1_01, file = f)
# with open(output_file_name, "a") as f: print('F1 xgboost: ', xgb_f1_01, file = f)




In [79]:
# with open(output_file_name, "a") as f: print('-----------------------', file = f)
# with open(output_file_name, "a") as f: print('Summary', file = f)
# with open(output_file_name, "a") as f: print('-----------------------', file = f)
# with open(output_file_name, "a") as f: print('Level 00', file = f)

# with open(output_file_name, "a") as f: print('Accuracy ada: ', ada_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy dnn: ', dnn_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy svm: ', svm_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy knn: ', knn_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy mlp: ', mlp_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy lgbm: ', lgbm_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy rf: ', rf_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy cat: ', cat_acc_00, file = f)
# with open(output_file_name, "a") as f: print('Accuracy xgb: ', xgb_acc_00, file = f)


# # with open(output_file_name, "a") as f: print('Precision ada: ', ada_pre_00, file = f)
# # with open(output_file_name, "a") as f: print('Precision dnn: ', dnn_pre_00, file = f)
# # with open(output_file_name, "a") as f: print('Precision svm: ', svm_pre_00, file = f)
# # with open(output_file_name, "a") as f: print('Precision knn: ', knn_pre_00, file = f)
# # with open(output_file_name, "a") as f: print('Precision mlp: ', mlp_pre_00, file = f)
# # with open(output_file_name, "a") as f: print('Precision lgbm: ', lgbm_pre_00, file = f)
# # with open(output_file_name, "a") as f: print('Precision rf: ', rf_pre_00, file = f)
# # with open(output_file_name, "a") as f: print('Precision cat: ', cat_pre_00, file = f)
# # with open(output_file_name, "a") as f: print('Precision xgb: ', xgb_pre_00, file = f)

# # with open(output_file_name, "a") as f: print('Recall ada: ', ada_rec_00, file = f)
# # with open(output_file_name, "a") as f: print('Recall dnn: ', dnn_rec_00, file = f)
# # with open(output_file_name, "a") as f: print('Recall svm: ', svm_rec_00, file = f)
# # with open(output_file_name, "a") as f: print('Recall knn: ', knn_rec_00, file = f)
# # with open(output_file_name, "a") as f: print('Recall mlp: ', mlp_rec_00, file = f)
# # with open(output_file_name, "a") as f: print('Recall lgbm: ', lgbm_rec_00, file = f)
# # with open(output_file_name, "a") as f: print('Recall rf: ', rf_rec_00, file = f)
# # with open(output_file_name, "a") as f: print('Recall cat: ', cat_rec_00, file = f)
# # with open(output_file_name, "a") as f: print('Recall xgb: ', xgb_rec_00, file = f)

# # with open(output_file_name, "a") as f: print('F1 ada: ', ada_f1_00, file = f)
# # with open(output_file_name, "a") as f: print('F1 dnn: ', dnn_f1_00, file = f)
# # with open(output_file_name, "a") as f: print('F1 svm: ', svm_f1_00, file = f)
# # with open(output_file_name, "a") as f: print('F1 knn: ', knn_f1_00, file = f)
# # with open(output_file_name, "a") as f: print('F1 mlp: ', mlp_f1_00, file = f)
# # with open(output_file_name, "a") as f: print('F1 lgbm: ', lgbm_f1_00, file = f)
# # with open(output_file_name, "a") as f: print('F1 rf: ', rf_f1_00, file = f)
# # with open(output_file_name, "a") as f: print('F1 cat: ', cat_f1_00, file = f)
# # with open(output_file_name, "a") as f: print('F1 xgb: ', xgb_f1_00, file = f)


# with open(output_file_name, "a") as f: print('-----------------------', file = f)
# with open(output_file_name, "a") as f: print('Level 01', file = f)

# with open(output_file_name, "a") as f: print('Accuracy ada: ', ada_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy dnn: ', dnn_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy svm: ', svm_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy knn: ', knn_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy mlp: ', mlp_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy lgbm: ', lgbm_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy rf: ', rf_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy LR: ', lr_acc_01, file = f)
# # with open(output_file_name, "a") as f: print('Accuracy Voting: ', voting_acc, file = f)
# with open(output_file_name, "a") as f: print('Accuracy catboost: ', cat_acc_01, file = f)
# with open(output_file_name, "a") as f: print('Accuracy xgb: ', xgb_acc_01, file = f)

# # with open(output_file_name, "a") as f: print('Precision ada: ', ada_pre_01, file = f)
# # with open(output_file_name, "a") as f: print('Precision dnn: ', dnn_pre_01, file = f)
# # with open(output_file_name, "a") as f: print('Precision svm: ', svm_pre_01, file = f)
# # with open(output_file_name, "a") as f: print('Precision knn: ', knn_pre_01, file = f)
# # with open(output_file_name, "a") as f: print('Precision mlp: ', mlp_pre_01, file = f)
# # with open(output_file_name, "a") as f: print('Precision lgbm: ', lgbm_pre_01, file = f)
# # with open(output_file_name, "a") as f: print('Precision rf: ', rf_pre_01, file = f)
# # with open(output_file_name, "a") as f: print('Precision LR: ', lr_pre_01, file = f)
# # with open(output_file_name, "a") as f: print('Precision Voting: ', voting_pre, file = f)
# # with open(output_file_name, "a") as f: print('Precision catboosting: ', cat_pre_01, file = f)
# # with open(output_file_name, "a") as f: print('Precision xgboost: ', xgb_pre_01, file = f)

# # with open(output_file_name, "a") as f: print('Recall ada: ', ada_rec_01, file = f)
# # with open(output_file_name, "a") as f: print('Recall dnn: ', dnn_rec_01, file = f)
# # with open(output_file_name, "a") as f: print('Recall svm: ', svm_rec_01, file = f)
# # with open(output_file_name, "a") as f: print('Recall knn: ', knn_rec_01, file = f)
# # with open(output_file_name, "a") as f: print('Recall mlp: ', mlp_rec_01, file = f)
# # with open(output_file_name, "a") as f: print('Recall lgbm: ', lgbm_rec_01, file = f)
# # with open(output_file_name, "a") as f: print('Recall rf: ', rf_rec_01, file = f)
# # with open(output_file_name, "a") as f: print('Recall LR: ', lr_rec_01, file = f)
# # with open(output_file_name, "a") as f: print('Recall Voting: ', voting_rec, file = f)
# # with open(output_file_name, "a") as f: print('Recall catboosting: ', cat_rec_01, file = f)
# # with open(output_file_name, "a") as f: print('Recall xgboost: ', xgb_rec_01, file = f)

# # with open(output_file_name, "a") as f: print('F1 ada: ', ada_f1_01, file = f)
# # with open(output_file_name, "a") as f: print('F1 dnn: ', dnn_f1_01, file = f)
# # with open(output_file_name, "a") as f: print('F1 svm: ', svm_f1_01, file = f)
# # with open(output_file_name, "a") as f: print('F1 knn: ', knn_f1_01, file = f)
# # with open(output_file_name, "a") as f: print('F1 mlp: ', mlp_f1_01, file = f)
# # with open(output_file_name, "a") as f: print('F1 lgbm: ', lgbm_f1_01, file = f)
# # with open(output_file_name, "a") as f: print('F1 rf: ', rf_f1_01, file = f)
# # with open(output_file_name, "a") as f: print('F1 LR: ', lr_f1_01, file = f)
# # with open(output_file_name, "a") as f: print('F1 Voting: ', voting_f1, file = f)
# # with open(output_file_name, "a") as f: print('F1 catboosting: ', cat_f1_01, file = f)
# # with open(output_file_name, "a") as f: print('F1 xgboost: ', xgb_f1_01, file = f)




In [80]:

# import sklearn
# from sklearn.model_selection import train_test_split
# split = 0.7

# #AUC ROC
# #---------------------------------------------------------------------

# #AUCROC
# aucroc =[]
# y_array = [y_0,y_1,y_2,y_3,y_4]
# for j in range(0,len(y_array)):
#     # print(j)
#     #------------------------------------------------------------------------------------------------------------
#     X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y_array[j], train_size=split)
    
#     # evaluate the model

#     knn_clf.fit(X_train,y_train)
#     y_pred=knn_clf.predict(X_test) #These are the predicted output value
#     # y_pred = knn_clf.predict_proba(X_test)

    
#     y_scores = y_pred
#     y_true = y_test

#     # model = LGBMClassifier()
#     # model.fit(X_train, y_train)
#     # y_pred = model.predict(X_test)


#     y_scores = y_pred
#     y_true = y_test
    
#     # Calculate AUC-ROC score
#     auc_roc_score= roc_auc_score(y_true, y_scores,  average='weighted')  # Use 'micro' or 'macro' for different averaging strategies
#     # print("AUC-ROC Score class:", auc_roc_score)
#     aucroc.append(auc_roc_score)
#     #-------------------------------------------------------------------------------------------------------    -----
#     # Calculate the average
# average = sum(aucroc) / len(aucroc)

# # Display the result
# # with open(output_file_name, "a") as f:print("AUC ROC Average:", average, file = f)
# print("AUC ROC Average:", average)

# #End AUC ROC

In [81]:
# lr_acc_00 = 0 
# voting_acc_00 = 0

# lr_pre_00 = 0 
# voting_pre_00 = 0

# lr_rec_00 = 0 
# voting_rec_00 = 0

# lr_f1_00 = 0 
# voting_f1_00 = 0

In [82]:

voting_acc_01 = 0


voting_pre_01 = 0


voting_rec_01 = 0


voting_f1_01 = 0

In [83]:
from tabulate import tabulate

# Assuming data is a 110x4 list, where each row is a sublist
# data =  [["Row {} Col {}".format(i + 1, j + 1) for j in range(4)] for i in range(110)]
data = [["" for _ in range(5)] for _ in range(12)]

# Manually insert data at specific row and column
# data[0][0] = "ADA"
# data[1][0] = "DNN"
# data[2][0] = "SVM"
# data[3][0] = "ADA"
# data[4][0] = "DNN"
# data[2][0] = "SVM"


names_models = ['ADA',
                'SVM',
                'DNN',
                'MLP',
                'KNN',
                'CAT',
                'XGB',
                'LGBM',
                'RF',
                'LR',
                'VOTING'
                ]
level_01_acc = [ada_acc_01,
                svm_acc_01,
                dnn_acc_01,
                mlp_acc_01,
                knn_acc_01,
                cat_acc_01,
                xgb_acc_01,
                lgbm_acc_01,
                rf_acc_01,
                lr_acc_01,
                voting_acc_01]  
level_01_pre = [ada_pre_01,
                svm_pre_01,
                dnn_pre_01,
                mlp_pre_01,
                knn_pre_01,
                cat_pre_01,
                xgb_pre_01,
                lgbm_pre_01,
                rf_pre_01,
                lr_pre_01,
                voting_pre_01] 
level_01_rec = [ada_rec_01,
                svm_rec_01,
                dnn_rec_01,
                mlp_rec_01,
                knn_rec_01,
                cat_rec_01,
                xgb_rec_01,
                lgbm_rec_01,
                rf_rec_01,
                lr_rec_01,
                voting_rec_01]  
level_01_f1 = [ada_f1_01,
                svm_f1_01,
                dnn_f1_01,
                mlp_f1_01,
                knn_f1_01,
                cat_f1_01,
                xgb_f1_01,
                lgbm_f1_01,
                rf_f1_01,
                lr_f1_01,
                voting_f1_01]  

                 

for i in range(0,len(names_models)):
    data[i][0] =  names_models[i]

    data[i][1] = level_01_acc[i]
    # data[i][2] = level_01_acc[i]

    data[i][2] = level_01_pre[i] 
    # data[i][4] = level_01_pre[i]

    data[i][3] = level_01_rec[i] 
    # data[i][6] = level_01_rec[i]

    data[i][4] = level_01_f1[i]
    # data[i][8] = level_01_f1[i]




 
# data[0][1] = ada_acc_01
# data

# Define column headers
# headers = ["Models", "ACC-01", " ACC-01","PRE-01", " PRE-01","REC-01", " REC-01","F1-01", " F1-01",]
headers = ["Models", "ACC-01","PRE-01","REC-01","F1-01"]


# Print the table
table = tabulate(data, headers=headers, tablefmt="grid")
print(table)
with open(output_file_name, "a") as f: print(table, file = f)


+----------+--------------------+--------------------+--------------------+--------------------+
| Models   | ACC-01             | PRE-01             | REC-01             | F1-01              |
+==========+====================+====================+====================+====================+
| ADA      | 0.9667833737319328 | 0.7452786169091856 | 0.7069300802727743 | 0.724398988880189  |
+----------+--------------------+--------------------+--------------------+--------------------+
| SVM      | 0.8792081874495018 | 0.5783299354191134 | 0.5599734594264867 | 0.5135936316789838 |
+----------+--------------------+--------------------+--------------------+--------------------+
| DNN      | 0.9430379746835443 | 0.5325704376912934 | 0.5579176145445526 | 0.5446612807463953 |
+----------+--------------------+--------------------+--------------------+--------------------+
| MLP      | 0.9811473202262322 | 0.7540175171198078 | 0.7632528959239488 | 0.7585292769243145 |
+----------+------------------

In [84]:
print('---------------------------------------------------------------------------------')
print('Generating SHAP explanation')
print('---------------------------------------------------------------------------------')
print('')
with open(output_file_name, "a") as f:print('ADA FEATURE IMPORTANCE',file = f)

    #START TIMER MODEL
start = time.time()

print('---------------------------------------------------------------------------------')
print('Generating explainer')
print('---------------------------------------------------------------------------------')
print('')
test = X_test_01
train = X_train_01
# ## Summary Bar Plot Global
start_index = 0
end_index = 250
# test.pop('Label')
# test.pop('is_train')
# print(label2)


# models = [ada,dnn_01,clf,knn_clf_01,cat_01,xgb_01, rf, lgbm, mlp,logreg_01]
explainer = shap.KernelExplainer(ada.predict_proba, test[start_index:end_index])

shap_values = explainer.shap_values(test[start_index:end_index])

shap.summary_plot(shap_values = shap_values,
                  features = test[start_index:end_index],
                  # class_names=[column_features[:-1]],
                  show=False)

# if feature_selection_bit == 1 # On
# pick_prob = 0 # set equal one to choose the dataset with probabilities, set to 0 to choose one with the classes.
if pick_prob == 1:
  plt.savefig('ADA_SHAP_NSL_prob_01.png')
elif pick_prob == 0:
  plt.savefig('ADA_SHAP_NSL_class_01.png')

else: None
plt.clf()


vals= np.abs(shap_values).mean(1)
feature_importance = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
feature_importance.head()
print(feature_importance.to_string())

with open(output_file_name, "a") as f:print('Feature Importance: ',feature_importance.to_string(),file = f)



end = time.time()
with open(output_file_name, "a") as f:print('ELAPSE TIME LIME GLOBAL: ',(end - start)/60, 'min',file = f)
print('---------------------------------------------------------------------------------')



print('---------------------------------------------------------------------------------')
# feature_importance_vals = 'feature_importance_vals'  # Replace with the name of the column you want to extract
feature_val = feature_importance['feature_importance_vals'].tolist()

# col_name = 'col_name'  # Replace with the name of the column you want to extract
feature_name = feature_importance['col_name'].tolist()


# for item1, item2 in zip(feature_name, feature_val):
#     print(item1, item2)


# Use zip to combine the two lists, sort based on list1, and then unzip them
zipped_lists = list(zip(feature_name, feature_val))
zipped_lists.sort(key=lambda x: x[1],reverse=True)

# Convert the sorted result back into separate lists
sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]

# for k in sorted_list1:
#   with open(output_file_name, "a") as f: print("df.pop('",k,"')", sep='', file = f)

# with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
# for k in sorted_list1:
#   with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
# with open(output_file_name, "a") as f:print("]", file = f)

print('---------------------------------------------------------------------------------')




Using 250 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


---------------------------------------------------------------------------------
Generating SHAP explanation
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Generating explainer
---------------------------------------------------------------------------------



  3%|▎         | 8/250 [00:00<00:14, 17.21it/s]

100%|██████████| 250/250 [00:13<00:00, 18.39it/s]


  col_name  feature_importance_vals
2     lgbm                 0.034430
1       rf                 0.011712
3      xgb                 0.007331
0      dnn                 0.006204
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------


<Figure size 576x223.2 with 0 Axes>

In [85]:

# explainer = shap.TreeExplainer(model)
# start_index = 0
# end_index = samples
# shap_values = explainer.shap_values(test[start_index:end_index])
# shap_obj = explainer(test[start_index:end_index])
# shap.summary_plot(shap_values = shap_values,
#                   features = test[start_index:end_index],
#                 show=False)
# plt.savefig('Light_SHAP_CIC_Summary.png')
# plt.clf()


# vals= np.abs(shap_values).mean(1)
# feature_importance = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
# feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
# feature_importance.head()
# print(feature_importance.to_string())
# print('---------------------------------------------------------------------------------')
# # feature_importance_vals = 'feature_importance_vals'  # Replace with the name of the column you want to extract
# feature_val = feature_importance['feature_importance_vals'].tolist()

# # col_name = 'col_name'  # Replace with the name of the column you want to extract
# feature_name = feature_importance['col_name'].tolist()


# # for item1, item2 in zip(feature_name, feature_val):
# #     print(item1, item2)


# # Use zip to combine the two lists, sort based on list1, and then unzip them
# zipped_lists = list(zip(feature_name, feature_val))
# zipped_lists.sort(key=lambda x: x[1],reverse=True)

# # Convert the sorted result back into separate lists
# sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]

# for k in sorted_list1:
#   with open(output_file_name, "a") as f:print("df.pop('",k,"')", sep='', file = f)

# # with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
# for k in sorted_list1:
#   with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
# with open(output_file_name, "a") as f:print("]", file = f)
# print('---------------------------------------------------------------------------------')


In [86]:
print('---------------------------------------------------------------------------------')
print('Generating SHAP explanation')
print('---------------------------------------------------------------------------------')
print('')

with open(output_file_name, "a") as f:print('XGB FEATURE IMPORTANCE',file = f)

    #START TIMER MODEL
start = time.time()

print('---------------------------------------------------------------------------------')
print('Generating explainer')
print('---------------------------------------------------------------------------------')
print('')
test = X_test_01
train = X_train_01
# ## Summary Bar Plot Global
start_index = 0
end_index = 250
# test.pop('Label')
# test.pop('is_train')
# print(label2)


explainer = shap.TreeExplainer(xgb_01)

shap_values = explainer.shap_values(test[start_index:end_index])
shap_obj = explainer(test[start_index:end_index])
shap.summary_plot(shap_values = shap_values,
                  features = test[start_index:end_index],
                show=False)
# plt.clf()

# if feature_selection_bit == 1 # On
# pick_prob = 0 # set equal one to choose the dataset with probabilities, set to 0 to choose one with the classes.
if pick_prob == 1:
  plt.savefig('XGB_SHAP_NSL_prob_01.png')
elif pick_prob == 0:
  plt.savefig('XGB_SHAP_NSL_class_01.png')

else: None
plt.clf()


vals= np.abs(shap_values).mean(1)
feature_importance = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
feature_importance.head()
print(feature_importance.to_string())
with open(output_file_name, "a") as f:print('Feature Importance: ',feature_importance.to_string(),file = f)



end = time.time()
with open(output_file_name, "a") as f:print('ELAPSE TIME LIME GLOBAL: ',(end - start)/60, 'min',file = f)
print('---------------------------------------------------------------------------------')



print('---------------------------------------------------------------------------------')
# feature_importance_vals = 'feature_importance_vals'  # Replace with the name of the column you want to extract
feature_val = feature_importance['feature_importance_vals'].tolist()

# col_name = 'col_name'  # Replace with the name of the column you want to extract
feature_name = feature_importance['col_name'].tolist()


# for item1, item2 in zip(feature_name, feature_val):
#     print(item1, item2)


# Use zip to combine the two lists, sort based on list1, and then unzip them
zipped_lists = list(zip(feature_name, feature_val))
zipped_lists.sort(key=lambda x: x[1],reverse=True)

# Convert the sorted result back into separate lists
sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]

# for k in sorted_list1:
#   with open(output_file_name, "a") as f: print("df.pop('",k,"')", sep='', file = f)

# with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
# for k in sorted_list1:
#   with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
# with open(output_file_name, "a") as f:print("]", file = f)

print('---------------------------------------------------------------------------------')




---------------------------------------------------------------------------------
Generating SHAP explanation
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Generating explainer
---------------------------------------------------------------------------------

  col_name  feature_importance_vals
2     lgbm                 6.412169
1       rf                 2.172974
3      xgb                 2.030175
0      dnn                 1.079601
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------


<Figure size 576x223.2 with 0 Axes>

In [87]:
print('---------------------------------------------------------------------------------')
print('Generating SHAP explanation')
print('---------------------------------------------------------------------------------')
print('')

with open(output_file_name, "a") as f:print('LGBM FEATURE IMPORTANCE',file = f)

    #START TIMER MODEL
start = time.time()

print('---------------------------------------------------------------------------------')
print('Generating explainer')
print('---------------------------------------------------------------------------------')
print('')
test = X_test_01
train = X_train_01
# ## Summary Bar Plot Global
start_index = 0
end_index = 250
# test.pop('Label')
# test.pop('is_train')
# print(label2)


explainer = shap.TreeExplainer(lgbm)

shap_values = explainer.shap_values(test[start_index:end_index])
shap_obj = explainer(test[start_index:end_index])
shap.summary_plot(shap_values = shap_values,
                  features = test[start_index:end_index],
                show=False)
# plt.clf()

# if feature_selection_bit == 1 # On
# pick_prob = 0 # set equal one to choose the dataset with probabilities, set to 0 to choose one with the classes.
if pick_prob == 1:
  plt.savefig('LGBM_SHAP_NSL_prob_01.png')
elif pick_prob == 0:
  plt.savefig('LGBM_SHAP_NSL_class_01.png')

else: None
plt.clf()


vals= np.abs(shap_values).mean(1)
feature_importance = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
feature_importance.head()
print(feature_importance.to_string())
with open(output_file_name, "a") as f:print('Feature Importance: ',feature_importance.to_string(),file = f)



end = time.time()
with open(output_file_name, "a") as f:print('ELAPSE TIME LIME GLOBAL: ',(end - start)/60, 'min',file = f)
print('---------------------------------------------------------------------------------')



print('---------------------------------------------------------------------------------')
# feature_importance_vals = 'feature_importance_vals'  # Replace with the name of the column you want to extract
feature_val = feature_importance['feature_importance_vals'].tolist()

# col_name = 'col_name'  # Replace with the name of the column you want to extract
feature_name = feature_importance['col_name'].tolist()


# for item1, item2 in zip(feature_name, feature_val):
#     print(item1, item2)


# Use zip to combine the two lists, sort based on list1, and then unzip them
zipped_lists = list(zip(feature_name, feature_val))
zipped_lists.sort(key=lambda x: x[1],reverse=True)

# Convert the sorted result back into separate lists
sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]

# for k in sorted_list1:
#   with open(output_file_name, "a") as f: print("df.pop('",k,"')", sep='', file = f)

# with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
# for k in sorted_list1:
#   with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
# with open(output_file_name, "a") as f:print("]", file = f)

print('---------------------------------------------------------------------------------')




---------------------------------------------------------------------------------
Generating SHAP explanation
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Generating explainer
---------------------------------------------------------------------------------



  col_name  feature_importance_vals
2     lgbm                 9.598010
1       rf                 4.530714
0      dnn                 2.256583
3      xgb                 1.921904
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------


<Figure size 576x223.2 with 0 Axes>

In [88]:
print('---------------------------------------------------------------------------------')
print('Generating SHAP explanation')
print('---------------------------------------------------------------------------------')
print('')

with open(output_file_name, "a") as f:print('RF FEATURE IMPORTANCE',file = f)

    #START TIMER MODEL
start = time.time()

print('---------------------------------------------------------------------------------')
print('Generating explainer')
print('---------------------------------------------------------------------------------')
print('')
test = X_test_01
train = X_train_01
# ## Summary Bar Plot Global
start_index = 0
end_index = 250
# test.pop('Label')
# test.pop('is_train')
# print(label2)


explainer = shap.TreeExplainer(rf)

shap_values = explainer.shap_values(test[start_index:end_index])
shap_obj = explainer(test[start_index:end_index])
shap.summary_plot(shap_values = shap_values,
                  features = test[start_index:end_index],
                show=False)
# plt.clf()

# if feature_selection_bit == 1 # On
# pick_prob = 0 # set equal one to choose the dataset with probabilities, set to 0 to choose one with the classes.
if pick_prob == 1:
  plt.savefig('RF_SHAP_NSL_prob_01.png')
elif pick_prob == 0:
  plt.savefig('RF_SHAP_NSL_class_01.png')

else: None
plt.clf()


vals= np.abs(shap_values).mean(1)
feature_importance = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
feature_importance.head()
print(feature_importance.to_string())
with open(output_file_name, "a") as f:print('Feature Importance: ',feature_importance.to_string(),file = f)



end = time.time()
with open(output_file_name, "a") as f:print('ELAPSE TIME LIME GLOBAL: ',(end - start)/60, 'min',file = f)
print('---------------------------------------------------------------------------------')



print('---------------------------------------------------------------------------------')
# feature_importance_vals = 'feature_importance_vals'  # Replace with the name of the column you want to extract
feature_val = feature_importance['feature_importance_vals'].tolist()

# col_name = 'col_name'  # Replace with the name of the column you want to extract
feature_name = feature_importance['col_name'].tolist()


# for item1, item2 in zip(feature_name, feature_val):
#     print(item1, item2)


# Use zip to combine the two lists, sort based on list1, and then unzip them
zipped_lists = list(zip(feature_name, feature_val))
zipped_lists.sort(key=lambda x: x[1],reverse=True)

# Convert the sorted result back into separate lists
sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]

# for k in sorted_list1:
#   with open(output_file_name, "a") as f: print("df.pop('",k,"')", sep='', file = f)

# with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
# for k in sorted_list1:
#   with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
# with open(output_file_name, "a") as f:print("]", file = f)

print('---------------------------------------------------------------------------------')




---------------------------------------------------------------------------------
Generating SHAP explanation
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Generating explainer
---------------------------------------------------------------------------------

  col_name  feature_importance_vals
2     lgbm                 0.670630
3      xgb                 0.418141
1       rf                 0.106782
0      dnn                 0.024627
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------


<Figure size 576x223.2 with 0 Axes>